In [ ]:
import numpy as np
import random
import numpy.linalg as npl
from matplotlib import pyplot as plt
import copy

In [ ]:
num_clients=1000
base_s=1
data_A=[]
data_b=[]
client_s=[]
iid=False
n=10
d=10
t_list=[]
x_star=np.zeros(n)
y_star=np.zeros(n)
lmbda=0.1
for i in range(num_clients):
    if iid:
        client_s.append(base_s)
    else:
        client_s.append(abs(np.random.uniform(0,2*base_s)))
    
    s=client_s[-1]
    
    b = np.random.normal(0,s,n)
    b=b-np.mean(b)
    
    #A = np.diag(np.random.uniform(0,1.,n))

    #A = np.diag(np.random.normal(0,s**2.,n))
    #t = 0.01#np.random.uniform(0,1.)
    t = np.random.uniform(0,0.1)
    A = np.eye(n)*t
    t_list.append(t)
#     a=np.random.normal(0,s**2.,n)
#     a[a<1]=1
#     A=np.diag(a)


    #A = np.random.uniform(0,1,(n,n))
    data_A.append(A)
    data_b.append(b)
    x_star+=A@b
    
x_star/=num_clients
t_list=np.array(t_list)
x_star=1/(t_list.T@t_list/num_clients+lmbda)*(x_star)
print(x_star)

for i in range(num_clients):
    y_star+=data_A[i]@x_star
y_star/=num_clients
print(y_star)
print(np.mean(t_list))
t=np.mean(t_list)

In [ ]:
select_client=50
num_epochs=200
#lmbda=10
lr=0.01
hlr=0.01
tau=500
inner_tau=50
T=10

In [ ]:
result_joint=[]
random.seed(1)
y = np.ones(n, dtype=np.float32)*10
x = np.ones(d, dtype=np.float32)*10
x_clients=np.zeros((select_client,d))
y_clients=np.zeros((select_client,n))
cur_lr=lr
cur_hlr=hlr
for _ in range(num_epochs):
#     cur_lr*=0.999
#     cur_hlr*=0.999
    client_idx = np.random.choice(range(num_clients), select_client, replace=False)
    
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
#         b = np.random.normal(0,base_s**2,n)
#         A = np.eye(n)*np.random.uniform(0,1.)
        # inner optimization
        yi=y.copy()
        xi=x.copy()
        
        for _ in range(T):
            for _ in range(inner_tau):
                yi+=cur_lr*(1./n)*( - yi + 0.5*b - 0.5*A@xi)
                xi-=cur_hlr*(1./n)*( - 0.5*yi.T@A + lmbda*xi)
#         for _ in range(tau):
#             xi-=hlr*(1./n)*( - 0.5*yi.T@A + lmbda*xi)
        x_clients[i]=xi
        y_clients[i]=yi
    #print(x_clients,y_clients)
    x=np.mean(x_clients,axis=0)
    y=np.mean(y_clients,axis=0)
    
    loss=0
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        loss += (1./n)*-0.5*( y.T@y - b@y + y.T@A@x)+lmbda*x.T@x
        
    loss/=select_client
    #result_joint.append([npl.norm(x-x_star)**2,npl.norm(y-y_star)**2,loss])
    result_joint.append([npl.norm(x)**2,npl.norm(y)**2,loss])
result_joint=np.array(result_joint)
plt.plot(result_joint[:,0])
plt.plot(result_joint[:,1])
plt.legend(['x','y'])
plt.grid('--')
plt.yscale("log")
print(result_joint[-1,:])

In [ ]:
result_fednest=[]
random.seed(1)
y = np.ones(n, dtype=np.float32)*10
x = np.ones(d, dtype=np.float32)*10
x_clients=np.zeros((select_client,d))
y_clients=np.zeros((select_client,n))
for _ in range(num_epochs):
    client_idx = np.random.choice(range(num_clients), select_client, replace=False)
    gyi_svrg = np.zeros(n, dtype=np.float32)*10
    gxi_svrg = np.zeros(d, dtype=np.float32)*10
    
    for _ in range(T):
        client_idx = np.random.choice(range(num_clients), select_client, replace=False)
        t_avg=0
        for i in range(select_client):
            client = client_idx[i]
            A=data_A[client]
            t_avg+=t_list[client]
        t_avg/=select_client
        for i in range(select_client):
            client = client_idx[i]
            A=data_A[client]
            b=data_b[client]
            yi=copy.deepcopy(y)
            xi=copy.deepcopy(x)
            for _ in range(inner_tau):            
                yi+=lr*(1./n)*(-yi-0.5*t_avg*x)
            y_clients[i]=yi
        y=np.mean(y_clients,axis=0)
    
    client_idx = np.random.choice(range(num_clients), select_client, replace=False)
    t_avg=0
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        t_avg+=t_list[client]
    t_avg/=select_client
        
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        yi=copy.deepcopy(y)
        xi=copy.deepcopy(x)
        for _ in range(tau):
            xi-=hlr*(1./n)*( lmbda*xi - 0.5*y*t_avg )
        x_clients[i]=xi
    x=np.mean(x_clients,axis=0)

    loss=0
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        loss += (1./n)*-0.5*( y.T@y - b@y + y.T@A@x)+lmbda*x.T@x
    loss/=select_client
    
    result_fednest.append([npl.norm(x)**2,npl.norm(y)**2,loss])
    #result_fednest.append([npl.norm(x-x_star)**2,npl.norm(y-y_star)**2,loss])
result_fednest=np.array(result_fednest)
plt.plot(result_fednest[:,0])
plt.plot(result_fednest[:,1])
plt.legend(['x','y'])
plt.grid('--')
plt.yscale("log")

In [ ]:
result_lfednest=[]
random.seed(1)
y = np.ones(n, dtype=np.float32)*10
x = np.ones(d, dtype=np.float32)*10
x_clients=np.zeros((select_client,d))
y_clients=np.zeros((select_client,n))
for _ in range(num_epochs):
    client_idx = np.random.choice(range(num_clients), select_client, replace=False)
    
    for _ in range(T):
        client_idx = np.random.choice(range(num_clients), select_client, replace=False)
        for i in range(select_client):
            client = client_idx[i]
            A=data_A[client]
            b=data_b[client]
            yi=y.copy()
            xi=x.copy()
            for _ in range(inner_tau):
                yi+=lr*0.1*( - yi + 0.5*b - 0.5*A@xi)
            y_clients[i]=yi
        y=np.mean(y_clients,axis=0)
    
    client_idx = np.random.choice(range(num_clients), select_client, replace=False)
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        yi=y.copy()
        xi=x.copy()
        for _ in range(tau):
            xi-=hlr*0.1*( - 0.5*yi.T@A + lmbda*xi)
        x_clients[i]=xi
    x=np.mean(x_clients,axis=0)

    loss=0
    for i in range(select_client):
        client = client_idx[i]
        A=data_A[client]
        b=data_b[client]
        loss += (1./n)*-0.5*( y.T@y - b@y + y.T@A@x)+lmbda*x.T@x

    loss/=select_client
    result_lfednest.append([npl.norm(x)**2,npl.norm(y)**2,loss])
    #result_lfednest.append([npl.norm(x-x_star)**2,npl.norm(y-y_star)**2,loss])
result_lfednest=np.array(result_lfednest)
plt.plot(result_lfednest[:,0])
plt.plot(result_lfednest[:,1])
plt.legend(['x','y'])
plt.grid('--')
plt.yscale("log")

In [ ]:
plt.plot(result_joint[:,0])
plt.plot(result_lfednest[:,0])
plt.plot(result_fednest[:,0])
plt.grid('--')
plt.legend(['joint','lfednest','fednest'])
plt.yscale('log')


In [ ]:
plt.plot(result_joint[:,1])
plt.plot(result_lfednest[:,1])
plt.plot(result_fednest[:,1])
plt.grid('--')
plt.legend(['joint','lfednest','fednest'])
plt.yscale('log')

In [ ]:
plt.plot(result_joint[:,2])
plt.plot(result_lfednest[:,2])
plt.plot(result_fednest[:,2])
plt.grid('--')
plt.legend(['joint','lfednest','fednest'])
plt.ylim(-0.2,0.2)

In [ ]:
np.save(f'output/{base_s}.npy',[result_joint,result_lfednest,result_fednest])

In [ ]:
plt.tight_layout()
j1,l1,f1=np.load('output/1.npy',allow_pickle=True)
j10,l10,f10=np.load('output/10.npy',allow_pickle=True)
plt.plot(j1[:,0],color="tab:blue",linewidth=3)
plt.plot(j10[:,0],'--',color="tab:blue",linewidth=3)

plt.plot(l1[:,0],color="tab:orange",linewidth=3)
plt.plot(l10[:,0],'--',color="tab:orange",linewidth=3)

plt.plot(f1[:,0],color="tab:green",linewidth=3)
plt.plot(f10[:,0],'--',color="tab:green",linewidth=3)
plt.grid('--')
#plt.legend(['FedAvg-S, s=1','FedAvg-S, s=10','LFedNest, s=1', 'LFedNest, s=10', 'FedNest, s=1', 'FedNest, s=10'],
#           fontsize=13)
plt.yscale('log')
plt.ylim(1e-22,1e4)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig('output/Fig4b_mm_syn_x.pdf')

In [ ]:
plt.tight_layout()
j1,l1,f1=np.load('output/1.npy',allow_pickle=True)
j10,l10,f10=np.load('output/10.npy',allow_pickle=True)
plt.plot(j1[:,1],color="tab:blue",linewidth=3)
plt.plot(j10[:,1],'--',color="tab:blue",linewidth=3)

plt.plot(l1[:,1],color="tab:orange",linewidth=3)
plt.plot(l10[:,1],'--',color="tab:orange",linewidth=3)

plt.plot(f1[:,1],color="tab:green",linewidth=3)
plt.plot(f10[:,1],'--',color="tab:green",linewidth=3)
plt.grid('--')
plt.legend(['FedAvg-S, s=1','FedAvg-S, s=10','LFedNest, s=1', 'LFedNest, s=10', 'FedNest, s=1', 'FedNest, s=10'],
           fontsize=15)
plt.yscale('log')
plt.ylim(1e-22,1e4)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()
plt.savefig('output/Fig4a_mm_syn_y.pdf')